https://colab.research.google.com/github/rickiepark/handson-ml3/blob/main/05_support_vector_machines.ipynb

In [1]:
import sys

assert sys.version_info >= (3, 7)

from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [2]:
from pathlib import Path

IMAGES_PATH = Path() / "images" / "svm"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 5.1 선형 SVM 분류

선형 서포트 벡터 머신support vector machine(SVM)은 두 클래스 사이를 최대한으로 경계 도로를 최대한 넓게 잡으려고 시도한다. 이때 두 클래스 사이에 놓을 수 있는 결정 경계 도로의 폭의 마진margin이라 하며, 마진을 최대로 하는 분류가 큰 마진 분류large margin classication이다.

아래 그림은 붓꽃 데이터셋을 대상으로 해서 선형 분류와 큰 마진 분류의 차이점을 보여준다. 선형 분류(왼쪽 그래프)의 경우 두 클래스를 분류하기만 해도 되는 반면에 큰 마진 분류(오른쪽 그래프)의 결정 경계(검은 실선)는 두 클래스와 거리를 최대한 크게 두려는 방향으로 정해진다. 즉, 마진은 가능한 최대로 유지하려 한다. 큰 마진 분류의 결정 경계는 결정 경계 도로의 가장자리에 위치한 서포트 벡터support vector에만 의존하며 다른 데이터와는 전혀 상관 없다. 아래 오른쪽 그래프에서 서포트 벡터는 동그라미로 감싸져 있다.

![image](https://github.com/user-attachments/assets/63b2181c-8995-446b-a75b-78bfd2efb9d5)

### 스케일링과 마진



특성의 스케일을 조정하면 결정 경계가 훨씬 좋아진다. 두 특성의 스케일에 차이가 많이 나는 경우(아래 왼쪽 그래프) 보다 표준화된 특성을 사용할 때(아래 오른쪽 그래프) 훨씬 좋은 결정 경계가 찾아진다.

![image](https://github.com/user-attachments/assets/f3841a6c-dc33-4ac5-a1b9-237174c95fff)


### 5.1.1 소프트 마진 분류

**하드 마진 분류**

모든 훈련 샘플이 도로 바깥쪽에 올바르게 분류되도록 하는 마진 분류가 하드 마진 분류hard margin classification이다. 하지만 두 클래스가 선형적으로 구분되는 경우에만 적용 가능하다.

또한 이상치에 매우 민감하다. 하나의 이상치가 추가되면 선형 분류가 불가능하거나(아래 왼편 그래프) 일반화가 매우 어려운 분류 모델(아래 오른편 그래프)이 얻어질 수 있다.

![image](https://github.com/user-attachments/assets/fc90ecb8-477d-45bd-8c86-11cb5beb60b4)

**소프트 마진 분류**

소프트 마진 분류soft margin classification는 어느 정도의 마진 오류를 허용하면서 결정 경계 도로의 폭을 최대로 하는 방향으로 유도한다. 마진 오류margin violations는 결정 경계 도로 위에 또는 결정 경계를 넘어 해당 클래스 반대편에 위치하는 샘플을 가리키며 소프트 마진 분류의 서포트 벡터를 구성한다.

예를 들어 꽃잎 길이와 너비 기준으로 붓꽃의 버지니카와 버시컬러 품종을 하드 마진 분류하기는 불가능하며, 아래 그래프에서처럼 어느 정도의 마진 오류를 허용해야 한다.

![image](https://github.com/user-attachments/assets/7b5eb522-288d-4540-804e-04616ffc5139)

In [3]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = load_iris(as_frame=True)
X = iris.data[["petal length (cm)", "petal width (cm)"]].values
y = (iris.target == 2)  # Iris virginica

svm_clf = make_pipeline(StandardScaler(),
                        LinearSVC(C=1, dual=True, random_state=42))
svm_clf.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(C=1, dual=True, random_state=42))])

In [4]:
X_new = [[5.5, 1.7], [5.0, 1.5]]
svm_clf.predict(X_new)

array([ True, False])

In [5]:
svm_clf.decision_function(X_new)

array([ 0.66163411, -0.22036063])

사이킷런의 `LinearSVC` 클래스는 선형 SVM 분류기를 생성한다.

`C` 는 규제 강조를 지정하는 하이퍼파라미터이며 클 수록 적은 규제를 의미한다. `C` 가 너무 작으면(아래 왼편 그래프) 마진 오류를 너무 많이 허용하는 과소 적합이 발생하며, `C` 를 키우면(아래 오른편 그래프) 결정 경계 도로 폭이 좁아진다. 여기서는 `C=100` 이 일반화 성능이 좋은 모델을 유도하는 것으로 보인다. 또한 `C=float("inf")`로 지정하면 하드 마진 분류 모델이 된다.

> `C=100`인 모델이 `C=1`인 모델보다 더 잘 일반화가 된 것이다.

![image](https://github.com/user-attachments/assets/41e269ee-b926-4d45-ab7d-0d8c3986f242)

# 5.2 비선형 SVM 분류

비선형 데이터를 다루는 한 가지 방법:

다항 특성과 같은 특성을 더 추가한다.

아래 외쪽 그래프는 하나의 특성 $x_1$만을 가진 간단한 데이터셋을 나타낸다.
- 이것은 선형적으로 구분할 수 없다.

하지만 두 번째 특성 $x_2 = (x_1)^2$을 추가하여 만들어진 2차원 데이터셋을 완변하게 선형적으로 구분한다.

![image](https://github.com/user-attachments/assets/be8fda61-6407-42d2-8324-10ff5ce595e2)

In [6]:
# 선형 SVM 분류 모델을 적용하기 위해 먼저 3차 항에 해당하는 특성을 추가하면 비선형 분류 모델을 얻게 된다.
from sklearn.datasets import make_moons
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(n_samples=100, noise=0.15, random_state=42)

polynomial_svm_clf = make_pipeline(
    PolynomialFeatures(degree=3),
    StandardScaler(),
    LinearSVC(C=10, max_iter=10_000, dual=True, random_state=42)
)
polynomial_svm_clf.fit(X, y)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('standardscaler', StandardScaler()),
                ('linearsvc',
                 LinearSVC(C=10, dual=True, max_iter=10000, random_state=42))])

![image](https://github.com/user-attachments/assets/747fcafa-2828-40b7-bdd7-1bd93e1f4765)

### 5.2.1 다항식 커널

다항 특성을 추가하는 것
- 간단
- 모든 머신러닝에서 잘 작동함

but!

단점이 있다:
- 낮은 차수의 다항식은 매우 복잡한 데이터셋을 잘 표현하지 못한다.
- 높은 차수의 다항식은 굉장히 많은 특성을 추가하므로 모델을 느리게 한다.



거의 기적에 가까운 수학적 기교를 적용한 `커널 트릭`을 이용하여 문제를 해소한다.

In [7]:
from sklearn.svm import SVC

poly_kernel_svm_clf = make_pipeline(StandardScaler(),
                                    SVC(kernel="poly", degree=3, coef0=1, C=5))
poly_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=5, coef0=1, kernel='poly'))])

![image](https://github.com/user-attachments/assets/7a87e05a-a9ed-4780-8cd6-ab6c77ff7117)


 **하이퍼파라미터 이해의 중요성**

> 다항 커널 모델이 과대 적합이면 차수를 줄여야 하고, 과소 적합이면 차수를 늘려야 한다. 적절한 하이퍼파라미터는 그리드 탐색 등을 이용하여 찾으면 되지만, 그럼에도 불구하고 하이퍼파라미터의 의미를 잘 알고 있으면 탐색 구간을 줄일 수 있다.

### 5.2.2 유사도 특성

가우스 방사 기저 함수를 이용한 유사도 특성similarity feature은 랜드마크landmark로 지정된 특정 샘플과 각 샘플이 얼마나 유사한가를 계산한다. 가우스 방사 기저 함수Gaussian radial basis function(Gaussian RBF)의 정의는 다음과 같다. 특정 지점을 가리키는 랜드마크landmark인 $m$으로부터 조금만 멀어져도 함숫값이 급격히 작아진다.

$$ϕ(X,m) = exp(-\gamma||X-m||^2)$$

![image](https://github.com/user-attachments/assets/98eaeb6f-9729-4fb6-8c14-2277e882a922)

하이퍼파라미터인 감마($\gamma$, gamma)는 데이터 샘플이 랜드마크로부터 멀어질 때 가우스 RBF 함수의 반환값이 얼마나 빠르게 0에 수렴하도록 하는가를 결정한다. 감마 값이 클 수록 랜드마크로부터 조금만 멀어져도 보다 빠르게 0에 수렴한다. 따라서 가우스 RBF 함수의 그래프가 보다 좁은 종 모양을 띤다.

예를 들어 아래 그래프는 감마가 1일 때와 0.01 때의 차이를 명확하게 보여준다. 즉 랜드마크인 $m = 0$으로부터 거리가 멀어질 때 감마가 1이면 매우 급격하게 함숫값이 0으로 줄어든다. 즉, 랜드마크로부터 조금만 멀어저도 유사도가 매우 빠르게 약해진다.

![image](https://github.com/user-attachments/assets/0cc939d8-bed9-4edd-be4f-49f2c075266c)

### 5.2.3 가우스 RBF 커널